In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
# --- PARTE 1: GERAR A BASE DE DADOS FICTÍCIA ---

print("--- Gerando base de dados histórica fictícia ---")

# Número de amostras na nossa base de dados
num_amostras = 10000

# Dicionário para criar as colunas do nosso DataFrame
dados = {
    'id_cliente': range(1, num_amostras + 1),
    'valor_divida_mil': np.random.lognormal(mean=3.5, sigma=1.5, size=num_amostras).round(2),
    'tempo_inadimplencia_dias': np.random.randint(30, 1800, size=num_amostras),
    'porte_cliente': np.random.choice(['Pequeno', 'Médio', 'Grande'], size=num_amostras, p=[0.5, 0.4, 0.1]),
    'ramo_atuacao_cliente': np.random.choice(['Produtor', 'Cooperativa', 'Distribuidor', 'Indústria'], size=num_amostras, p=[0.4, 0.3, 0.2, 0.1]),
    'score_risco_interno': np.random.choice(['A', 'B', 'C', 'D', 'F'], size=num_amostras, p=[0.1, 0.2, 0.4, 0.2, 0.1]),
    'regiao': np.random.choice(['Sul', 'Sudeste', 'Centro-Oeste', 'Nordeste', 'Norte'], size=num_amostras, p=[0.3, 0.3, 0.2, 0.1, 0.1])
}

df = pd.DataFrame(dados)

# --- Lógica para criar a variável-alvo (status_final_pago) ---
# Aqui está a "mágica": criamos uma probabilidade de pagamento baseada nas outras features.
# Isso garante que existam padrões a serem aprendidos.

# 1. Começamos com uma probabilidade base
prob_base = 0.5

# 2. Ajustamos a probabilidade com base nas regras de negócio
# Dívidas maiores e mais antigas são mais difíceis de cobrar
prob_base -= df['valor_divida_mil'] * 0.001
prob_base -= df['tempo_inadimplencia_dias'] * 0.0002

# Clientes maiores e com score melhor tendem a pagar mais
porte_map = {'Pequeno': -0.1, 'Médio': 0.05, 'Grande': 0.2}
score_map = {'A': 0.3, 'B': 0.15, 'C': 0.0, 'D': -0.2, 'F': -0.4}
prob_base += df['porte_cliente'].map(porte_map)
prob_base += df['score_risco_interno'].map(score_map)

# Adicionamos um pouco de aleatoriedade (ruído) para tornar mais realista
ruido = np.random.normal(0, 0.1, size=num_amostras)
prob_final = prob_base + ruido

# 3. Convertendo a probabilidade final (que pode ser > 1 ou < 0) para uma decisão binária (0 ou 1)
# Usamos a função sigmoide para garantir que a probabilidade esteja entre 0 e 1
prob_final_sigmoid = 1 / (1 + np.exp(-prob_final))
df['status_final_pago'] = (prob_final_sigmoid > np.random.rand(num_amostras)).astype(int)

# Salvar em um arquivo CSV para simular o recebimento do arquivo
caminho_arquivo = 'dividas_historicas_ficticias.csv'
df.to_csv(caminho_arquivo, index=False)

print(f"Base de dados com {num_amostras} clientes criada e salva em '{caminho_arquivo}'")
print("Distribuição da variável-alvo:")
print(df['status_final_pago'].value_counts(normalize=True))
print("\nPrimeiras 5 linhas da base de dados:")
print(df.head())


# --- PARTE 2: TREINAR O MODELO XGBOOST COM A BASE CRIADA ---

print("\n\n--- Treinando o modelo XGBoost com a base gerada ---")

# Carregar os dados (como se você tivesse recebido o arquivo)
df_treino = pd.read_csv(caminho_arquivo)

# 1. Preparação dos Dados (Pré-processamento)
# O XGBoost precisa que as variáveis categóricas (texto) sejam transformadas em números.
# Usamos "One-Hot Encoding" para isso, que cria colunas binárias para cada categoria.
df_treino_encoded = pd.get_dummies(df_treino, columns=['porte_cliente', 'ramo_atuacao_cliente', 'score_risco_interno', 'regiao'])

# Separar as features (X) da variável-alvo (y)
X = df_treino_encoded.drop(['id_cliente', 'status_final_pago'], axis=1)
y = df_treino_encoded['status_final_pago']

# 2. Dividir os dados em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# 3. Treinar o Modelo XGBoost
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc', use_label_encoder=False)
model.fit(X_train, y_train)

# 4. Fazer Predições e Avaliar
predictions_proba = model.predict_proba(X_test)[:, 1]
predictions_binary = model.predict(X_test)

accuracy = accuracy_score(y_test, predictions_binary)
auc = roc_auc_score(y_test, predictions_proba)

print(f"\nAcurácia do modelo no conjunto de teste: {accuracy * 100:.2f}%")
print(f"AUC do modelo no conjunto de teste: {auc:.4f}")

# 5. Analisar a Importância das Variáveis (Feature Importance)
# Isso nos mostra o que o modelo considerou mais importante para tomar a decisão.
# É a prova de que ele "descobriu" as regras que embutimos na geração dos dados!
print("\nImportância das variáveis (o que o modelo aprendeu):")
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
print(feature_importances.sort_values(by='importance', ascending=False).head(10))

# Plotar a importância das features para melhor visualização
fig, ax = plt.subplots(figsize=(10, 8))
xgb.plot_importance(model, ax=ax, max_num_features=15)
plt.title('Top 15 Variáveis Mais Importantes')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import joblib # Usaremos joblib para salvar e carregar o modelo

# --- ETAPA 1: SIMULAR O CENÁRIO PASSADO (TREINAMENTO DO MODELO) ---
# Esta parte gera a base histórica e treina o modelo, como fizemos antes.

print("--- ETAPA 1: Treinando o modelo com dados históricos ---")

# Gerar a base de dados histórica fictícia
num_amostras_treino = 10000
dados_treino = {
    'id_cliente': range(1, num_amostras_treino + 1),
    'valor_divida_mil': np.random.lognormal(mean=3.5, sigma=1.5, size=num_amostras_treino).round(2),
    'tempo_inadimplencia_dias': np.random.randint(30, 1800, size=num_amostras_treino),
    'porte_cliente': np.random.choice(['Pequeno', 'Médio', 'Grande'], size=num_amostras_treino, p=[0.5, 0.4, 0.1]),
    'ramo_atuacao_cliente': np.random.choice(['Produtor', 'Cooperativa', 'Distribuidor', 'Indústria'], size=num_amostras_treino, p=[0.4, 0.3, 0.2, 0.1]),
    'score_risco_interno': np.random.choice(['A', 'B', 'C', 'D', 'F'], size=num_amostras_treino, p=[0.1, 0.2, 0.4, 0.2, 0.1]),
    'regiao': np.random.choice(['Sul', 'Sudeste', 'Centro-Oeste', 'Nordeste', 'Norte'], size=num_amostras_treino, p=[0.3, 0.3, 0.2, 0.1, 0.1])
}
df_treino = pd.DataFrame(dados_treino)
prob_base = 0.5 - df_treino['valor_divida_mil'] * 0.001 - df_treino['tempo_inadimplencia_dias'] * 0.0002
porte_map = {'Pequeno': -0.1, 'Médio': 0.05, 'Grande': 0.2}
score_map = {'A': 0.3, 'B': 0.15, 'C': 0.0, 'D': -0.2, 'F': -0.4}
prob_base += df_treino['porte_cliente'].map(porte_map) + df_treino['score_risco_interno'].map(score_map)
prob_final_sigmoid = 1 / (1 + np.exp(-(prob_base + np.random.normal(0, 0.1, size=num_amostras_treino))))
df_treino['status_final_pago'] = (prob_final_sigmoid > np.random.rand(num_amostras_treino)).astype(int)

# Pré-processamento e Treinamento
df_treino_encoded = pd.get_dummies(df_treino, columns=['porte_cliente', 'ramo_atuacao_cliente', 'score_risco_interno', 'regiao'])
X_train = df_treino_encoded.drop(['id_cliente', 'status_final_pago'], axis=1)
y_train = df_treino_encoded['status_final_pago']
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc', use_label_encoder=False)
model.fit(X_train, y_train)

# Salvar o modelo treinado (o "manual de instruções")
caminho_modelo = 'modelo_score_recuperacao_v1.pkl'
joblib.dump(model, caminho_modelo)
print(f"Modelo treinado e salvo em '{caminho_modelo}'")


# --- ETAPA 2: SIMULAR A NOVA CARTEIRA E FAZER A PONTUAÇÃO (SCORING) ---

print("\n\n--- ETAPA 2: Gerando e avaliando a nova carteira de prospects ---")

# Gerar a nova base de dados para avaliação (prospects)
num_prospects = 2000
dados_prospects = {
    'id_cliente': range(20001, 20001 + num_prospects), # IDs diferentes para não confundir
    'valor_divida_mil': np.random.lognormal(mean=3.8, sigma=1.6, size=num_prospects).round(2), # Simula uma carteira ligeiramente diferente
    'tempo_inadimplencia_dias': np.random.randint(90, 1500, size=num_prospects),
    'porte_cliente': np.random.choice(['Pequeno', 'Médio', 'Grande'], size=num_prospects, p=[0.6, 0.3, 0.1]),
    'ramo_atuacao_cliente': np.random.choice(['Produtor', 'Cooperativa', 'Distribuidor', 'Indústria'], size=num_prospects, p=[0.5, 0.2, 0.2, 0.1]),
    'score_risco_interno': np.random.choice(['A', 'B', 'C', 'D', 'F'], size=num_prospects, p=[0.05, 0.15, 0.5, 0.2, 0.1]),
    'regiao': np.random.choice(['Sul', 'Sudeste', 'Centro-Oeste', 'Nordeste', 'Norte'], size=num_prospects, p=[0.2, 0.4, 0.25, 0.1, 0.05])
}
df_prospects = pd.DataFrame(dados_prospects)

# Salvar a carteira de prospects em um CSV
caminho_prospects = 'prospects_para_avaliar.csv'
df_prospects.to_csv(caminho_prospects, index=False)
print(f"Nova carteira com {num_prospects} prospects gerada e salva em '{caminho_prospects}'")

# Carregar o modelo que acabamos de salvar
modelo_carregado = joblib.load(caminho_modelo)

# Pré-processar os dados dos prospects EXATAMENTE da mesma forma que os dados de treino
df_prospects_encoded = pd.get_dummies(df_prospects, columns=['porte_cliente', 'ramo_atuacao_cliente', 'score_risco_interno', 'regiao'])
X_prospects = df_prospects_encoded.drop('id_cliente', axis=1)

# PONTO CRÍTICO: Garantir que as colunas da base de prospects sejam as mesmas da base de treino
# Se um prospect não tiver uma categoria que existia no treino (ex: nenhum da região 'Norte'),
# essa coluna não seria criada, causando um erro. O código abaixo alinha as colunas.
X_prospects_aligned = X_prospects.reindex(columns=X_train.columns, fill_value=0)

# Fazer a predição de probabilidade (o Score!)
scores_recuperacao = modelo_carregado.predict_proba(X_prospects_aligned)[:, 1]

# Adicionar os scores calculados à nossa base de prospects
df_prospects['score_recuperacao'] = scores_recuperacao

# --- ETAPA 3: ANALISAR O RESULTADO ---

print("\n\n--- ETAPA 3: Resultado da Análise da Carteira ---")

# Ordenar a carteira do maior para o menor score
df_resultado_final = df_prospects.sort_values(by='score_recuperacao', ascending=False)

print("\n--- Top 10 Prospects com MAIOR Chance de Recuperação ---")
print(df_resultado_final.head(10))

print("\n--- Top 10 Prospects com MENOR Chance de Recuperação ---")
print(df_resultado_final.tail(10))

# Calcular o Valor Esperado da carteira (soma das probabilidades x valor da dívida)
valor_esperado_total = (df_resultado_final['score_recuperacao'] * (df_resultado_final['valor_divida_mil'] * 1000)).sum()
df_resultado_final.to_csv("final.csv")

print(f"\n\nVALOR ESPERADO TOTAL DE RECUPERAÇÃO DA CARTEIRA: R$ {valor_esperado_total:,.2f}")

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import joblib # Usaremos joblib para salvar e carregar o modelo

# --- ETAPA 1: SIMULAR O CENÁRIO PASSADO (TREINAMENTO DO MODELO) ---
# Esta parte gera a base histórica e treina o modelo, como fizemos antes.

print("--- ETAPA 1: Treinando o modelo com dados históricos ---")

# Gerar a base de dados histórica fictícia
num_amostras_treino = 10000
dados_treino = {
    'id_cliente': range(1, num_amostras_treino + 1),
    'valor_divida_mil': np.random.lognormal(mean=3.5, sigma=1.5, size=num_amostras_treino).round(2),
    'tempo_inadimplencia_dias': np.random.randint(30, 1800, size=num_amostras_treino),
    'porte_cliente': np.random.choice(['Pequeno', 'Médio', 'Grande'], size=num_amostras_treino, p=[0.5, 0.4, 0.1]),
    'ramo_atuacao_cliente': np.random.choice(['Produtor', 'Cooperativa', 'Distribuidor', 'Indústria'], size=num_amostras_treino, p=[0.4, 0.3, 0.2, 0.1]),
    'score_risco_interno': np.random.choice(['A', 'B', 'C', 'D', 'F'], size=num_amostras_treino, p=[0.1, 0.2, 0.4, 0.2, 0.1]),
    'regiao': np.random.choice(['Sul', 'Sudeste', 'Centro-Oeste', 'Nordeste', 'Norte'], size=num_amostras_treino, p=[0.3, 0.3, 0.2, 0.1, 0.1])
}
df_treino = pd.DataFrame(dados_treino)
prob_base = 0.5 - df_treino['valor_divida_mil'] * 0.001 - df_treino['tempo_inadimplencia_dias'] * 0.0002
porte_map = {'Pequeno': -0.1, 'Médio': 0.05, 'Grande': 0.2}
score_map = {'A': 0.3, 'B': 0.15, 'C': 0.0, 'D': -0.2, 'F': -0.4}
prob_base += df_treino['porte_cliente'].map(porte_map) + df_treino['score_risco_interno'].map(score_map)
prob_final_sigmoid = 1 / (1 + np.exp(-(prob_base + np.random.normal(0, 0.1, size=num_amostras_treino))))
df_treino['status_final_pago'] = (prob_final_sigmoid > np.random.rand(num_amostras_treino)).astype(int)

# Pré-processamento e Treinamento
df_treino_encoded = pd.get_dummies(df_treino, columns=['porte_cliente', 'ramo_atuacao_cliente', 'score_risco_interno', 'regiao'])
X_train = df_treino_encoded.drop(['id_cliente', 'status_final_pago'], axis=1)
y_train = df_treino_encoded['status_final_pago']
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc', use_label_encoder=False)
model.fit(X_train, y_train)

artefatos_modelo = {
    'model': model,
    'columns': X_train.columns.tolist() # Salvamos a lista de nomes das colunas
}

# Salvar o dicionário (os "artefatos") em um único arquivo
caminho_modelo = 'modelo_score_recuperacao_with_columns_v1.pkl'
joblib.dump(artefatos_modelo, caminho_modelo) # Agora salvamos o dicionário

print(f"Artefatos do modelo (modelo + colunas) salvos em '{caminho_modelo}'")

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import plotly.express as px

# --- 1. SIMULAR UMA BASE DE DADOS COMO A SUA ---
# (Esta parte apenas cria um DataFrame para o exemplo)
# num_amostras = 1000
# dados = {
#     'id_cliente': range(1, num_amostras + 1),
#     'valor_divida_mil': np.random.lognormal(mean=3.5, sigma=1.5, size=num_amostras).round(2),
#     'tempo_inadimplencia_dias': np.random.randint(30, 1800, size=num_amostras),
#     'porte_cliente': np.random.choice(['Pequeno', 'Médio', 'Grande'], size=num_amostras, p=[0.5, 0.4, 0.1]),
#     'ramo_atuacao_cliente': np.random.choice(['Produtor', 'Cooperativa', 'Distribuidor', 'Indústria'], size=num_amostras),
#     'score_risco_interno': np.random.choice(['A', 'B', 'C', 'D', 'F'], size=num_amostras),
#     'regiao': np.random.choice(['Sul', 'Sudeste', 'Centro-Oeste', 'Nordeste', 'Norte'], size=num_amostras)
# }
# df = pd.DataFrame(dados)

df = pd.read_csv("prospects_para_avaliar.csv")


# --- 2. PRÉ-PROCESSAMENTO ---

# Separar colunas numéricas e categóricas
df_features = df.drop('id_cliente', axis=1)
colunas_numericas = ['valor_divida_mil', 'tempo_inadimplencia_dias']
colunas_categoricas = ['porte_cliente', 'ramo_atuacao_cliente', 'score_risco_interno', 'regiao']

# One-Hot Encoding para as colunas categóricas
df_categoricas_encoded = pd.get_dummies(df_features[colunas_categoricas])

# Escalonamento (Standardization) para as colunas numéricas
scaler = StandardScaler()
df_numericas_scaled = pd.DataFrame(scaler.fit_transform(df_features[colunas_numericas]), columns=colunas_numericas)

# Juntar as features processadas em um único DataFrame
df_processado = pd.concat([df_numericas_scaled, df_categoricas_encoded], axis=1)

print("Formato dos dados após pré-processamento:", df_processado.shape)
print("Amostra dos dados prontos para o PCA:")
print(df_processado.head())


# --- 3. APLICANDO O PCA ---

# Instanciar o PCA. n_components define quantos componentes queremos.
# Se deixarmos em branco, ele calcula todos os possíveis.
pca = PCA()

# Aplicar o PCA aos dados processados
componentes_principais = pca.fit_transform(df_processado)

# Criar um DataFrame com os componentes principais
df_pca = pd.DataFrame(data = componentes_principais, columns = [f'PC_{i+1}' for i in range(df_processado.shape[1])])

print("\n--- Resultado do PCA ---")
print(df_pca.head())


# --- 4. ANALISANDO O RESULTADO ---

# O atributo 'explained_variance_ratio_' nos diz quanto da variância
# total dos dados cada componente principal consegue explicar.
variancia_explicada = pca.explained_variance_ratio_
variancia_acumulada = np.cumsum(variancia_explicada)

print(f"\nVariância explicada por cada componente: {np.round(variancia_explicada, 3)}")
print(f"Variância acumulada: {np.round(variancia_acumulada, 3)}")

# Geralmente, escolhemos o número de componentes que explica 95% da variância.
n_componentes_95 = np.where(variancia_acumulada >= 0.95)[0][0] + 1
print(f"\nNúmero de componentes para explicar 95% da variância: {n_componentes_95}")


# --- 5. VISUALIZAÇÃO COM OS DOIS PRIMEIROS COMPONENTES ---

# Adicionar os dois primeiros componentes ao DataFrame original para colorir o gráfico
df['PC_1'] = df_pca['PC_1']
df['PC_2'] = df_pca['PC_2']

# Criar um gráfico de dispersão interativo
fig = px.scatter(
    df,
    x='PC_1',
    y='PC_2',
    color='score_risco_interno', # Colorir os pontos pelo score de risco
    hover_data=['id_cliente', 'valor_divida_mil'],
    title='Visualização de Clientes com os 2 Primeiros Componentes Principais'
)
fig.show()

Formato dos dados após pré-processamento: (2000, 19)
Amostra dos dados prontos para o PCA:
   valor_divida_mil  tempo_inadimplencia_dias  porte_cliente_Grande  \
0         -0.259533                  0.745284                 False   
1         -0.156550                  0.075281                 False   
2          0.113582                  1.572508                 False   
3          0.559595                  0.532431                 False   
4         -0.330734                  0.370373                 False   

   porte_cliente_Médio  porte_cliente_Pequeno  \
0                False                   True   
1                False                   True   
2                False                   True   
3                 True                  False   
4                False                   True   

   ramo_atuacao_cliente_Cooperativa  ramo_atuacao_cliente_Distribuidor  \
0                             False                              False   
1                             False    